In [24]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [25]:
#ee.Authenticate()
#geemap.update_package()

ee.Initialize()

Map = geemap.Map(center=[31.539096,-81.422318], zoom=10)

ga_pt = ee.Feature(ee.Geometry.Point([-81.422318, 31.539096]))

ga_pts = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point([-81.422318, 31.539096]), {'site_id': '1'}),
  ee.Feature(ee.Geometry.Point([-81.295564, 31.535967]), {'site_id': '2'}),
  ee.Feature(ee.Geometry.Point([-81.229141, 31.518847]), {'site_id': '3'}),
  ee.Feature(ee.Geometry.Point([-81.365805, 31.451791]), {'site_id': '4'}),
  ee.Feature(ee.Geometry.Point([-81.340722, 31.435814]), {'site_id': '5'}),
  ee.Feature(ee.Geometry.Point([-81.280011, 31.387962]), {'site_id': '6'}),
  ee.Feature(ee.Geometry.Point([-81.478498, 31.334813]), {'site_id': '7'}),
  ee.Feature(ee.Geometry.Point([-81.415535, 31.30824]), {'site_id': '8'}),
  ee.Feature(ee.Geometry.Point([-81.333707, 31.351518]), {'site_id': '9'}),
  ee.Feature(ee.Geometry.Point([-81.270863, 31.476948]), {'site_id': '10'})
])

##Adding every plot coordinate
allplots_fc = 'C:/Users/arj26323/Documents/Data/Biomass datasets/Sapelo/GA_allplots_NEW.csv'
fc_all = geemap.csv_to_ee(allplots_fc, latitude = "Latitude", longitude = "Longitude")

In [26]:
##FLATS
def addFLATS(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B6'),
            'RED': image.select('B4'),
            'GREEN': image.select('B3')
        })
    
    return image.addBands(flats.rename('flats'))

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B5'),
            'RED': image.select('B3'),
            'GREEN': image.select('B2')
        })
    
    return image.addBands(flats.rename('flats'))

In [27]:
##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.
def maskL5sr(image):
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5

  qa = image.select('pixel_qa')

  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))

  return image.updateMask(mask).divide(10000) \
      .select("B[0-9]*") \
      .copyProperties(image, ["system:time_start"])

In [64]:
#Coincident imagery
l8_col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filter(ee.Filter.calendarRange(2012, 2023,'year')) \
    .filterBounds(ga_pts)

l7_col = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
    .filter(ee.Filter.calendarRange(1998, 2013,'year')) \
    .filterBounds(ga_pts)

l5_col = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filter(ee.Filter.calendarRange(1998, 2012,'year')) \
    .filterBounds(ga_pts)

l9_col = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2") \
    .filter(ee.Filter.calendarRange(2020, 2023,'year')) \
    .filterBounds(ga_pts)
##ALL MIGRATED TO COLLECTION 2!

In [42]:
print(l5_col.size().getInfo())
print(l7_col.size().getInfo())
print(l8_col.size().getInfo())
print(l9_col.size().getInfo())

505
576
569
55


In [30]:
#Filter dates to get coincident images between l5 and L7; L7 and L8; and L8 and L9
#https://gis.stackexchange.com/questions/386827/filter-two-image-collections-by-the-same-aqusition-date-in-google-earth-engine

def datefunc(image):
    date = ee.Date(image.get('system:time_start')).format("YYYY-MM-dd")
    date = ee.Date(date)
    return image.set('date', date)

x_8 = l8_col.map(datefunc)
x_7 = l7_col.map(datefunc)
x_5 = l5_col.map(datefunc)
x_9 = l9_col.map(datefunc)

filterTimeEq = ee.Filter.equals(
    leftField='date',
    rightField='date'
)

simpleJoin = ee.Join.simple()

#Shared_5/7
L57 = ee.ImageCollection(simpleJoin.apply(x_5, x_7, filterTimeEq))

#Shared_7/8
L78 = ee.ImageCollection(simpleJoin.apply(x_7, x_8, filterTimeEq))

#Shared_8/9
L89 = ee.ImageCollection(simpleJoin.apply(x_8, x_9, filterTimeEq))

#https://developers.google.com/earth-engine/guides/joins_simple

In [31]:
print('Landsat 5/7 coincident images: ' + str(L57.size().getInfo()))
print('Landsat 7/8 coincident images: ' + str(L78.size().getInfo()))
print('Landsat 8/9 coincident images: ' + str(L89.size().getInfo()))

L89.aggregate_array('system:id').getInfo()

#Only ONE instance of same-day imagery for all 3 sensors over the 20+-year period

Landsat 5/7 coincident images: 0
Landsat 7/8 coincident images: 0
Landsat 8/9 coincident images: 2


['LANDSAT/LC08/C02/T1_L2/LC08_016038_20211112',
 'LANDSAT/LC08/C02/T1_L2/LC08_016039_20211112']

In [62]:
#Near coincident imagery - within one day (or 86400000 milliseconds)

filterNear = ee.Filter.maxDifference(
    difference = 86400000,
    leftField='system:time_start',
    rightField='system:time_start'
)

N57 = ee.ImageCollection(simpleJoin.apply(l5_col, l7_col, filterNear))
N78 = ee.ImageCollection(simpleJoin.apply(l7_col, l8_col, filterNear))
N89 = ee.ImageCollection(simpleJoin.apply(l8_col, l9_col, filterNear))

print('Landsat 5/7 NEAR coincident images: ' + str(N57.size().getInfo()))
print('Landsat 7/8 NEAR coincident images: ' + str(N78.size().getInfo()))
print('Landsat 8/9 NEAR coincident images: ' + str(N89.size().getInfo()))


Landsat 5/7 NEAR coincident images: 328
Landsat 7/8 NEAR coincident images: 19
Landsat 8/9 NEAR coincident images: 52


In [63]:
# N57.aggregate_array('system:id').getInfo()
N78.aggregate_array('system:id').getInfo()
# N89.aggregate_array('system:id').getInfo()

['LANDSAT/LE07/C02/T1_L2/LE07_016038_20130623',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20130709',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20130725',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20130810',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20130826',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20130911',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20131013',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20131029',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20131114',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20131130',
 'LANDSAT/LE07/C02/T1_L2/LE07_017038_20130411',
 'LANDSAT/LE07/C02/T1_L2/LE07_017038_20130427',
 'LANDSAT/LE07/C02/T1_L2/LE07_017038_20130513',
 'LANDSAT/LE07/C02/T1_L2/LE07_017038_20131004',
 'LANDSAT/LE07/C02/T1_L2/LE07_017038_20131020',
 'LANDSAT/LE07/C02/T1_L2/LE07_017038_20131105',
 'LANDSAT/LE07/C02/T1_L2/LE07_017038_20131121',
 'LANDSAT/LE07/C02/T1_L2/LE07_017038_20131207',
 'LANDSAT/LE07/C02/T1_L2/LE07_017038_20131223']

In [37]:
#Check coincident imagery:
check8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filter(ee.Filter.calendarRange(2021, 2021,'year')) \
    .filter(ee.Filter.calendarRange(11, 11,'month')) \
    .filterBounds(ga_pts)

check9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filter(ee.Filter.calendarRange(2021, 2021,'year')) \
    .filter(ee.Filter.calendarRange(11, 11,'month')) \
    .filterBounds(ga_pts)

print(check8.aggregate_array('system:id').getInfo())
print(check9.aggregate_array('system:id').getInfo())

['LANDSAT/LC08/C02/T1_L2/LC08_016038_20211112', 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20211128', 'LANDSAT/LC08/C02/T1_L2/LC08_016039_20211112', 'LANDSAT/LC08/C02/T1_L2/LC08_016039_20211128', 'LANDSAT/LC08/C02/T1_L2/LC08_017038_20211103', 'LANDSAT/LC08/C02/T1_L2/LC08_017038_20211119']
['LANDSAT/LC09/C02/T1_L2/LC09_016038_20211102', 'LANDSAT/LC09/C02/T1_L2/LC09_017038_20211107', 'LANDSAT/LC09/C02/T1_L2/LC09_017038_20211112']


In [38]:
# l7_col.aggregate_array('system:id').getInfo()
# l5_col.aggregate_array('system:id').getInfo()
l8_col.aggregate_array('system:id').getInfo()
# l9_col.aggregate_array('system:id').getInfo()

['LANDSAT/LC08/C02/T1_L2/LC08_016038_20130328',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20130412',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20130428',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20130514',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20130530',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20130615',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20130701',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20130717',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20130802',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20130818',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20130903',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20130919',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20131005',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20131021',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20131106',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20131122',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20131224',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20140109',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20140125',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_20140210',
 'LANDSAT/LC08/C02/T1_L2/LC08_016038_201

In [56]:
l7_col.aggregate_array('system:id').getInfo()
# l5_col.aggregate_array('system:id').getInfo()
# l8_col.aggregate_array('system:id').getInfo()
# l9_col.aggregate_array('system:id').getInfo()

['LANDSAT/LE07/C02/T1_L2/LE07_016038_19990703',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_19990719',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_19990804',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_19990820',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_19990905',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_19990921',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_19991007',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_19991023',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_19991108',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_19991124',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_19991210',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_19991226',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20000111',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20000127',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20000212',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20000228',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20000315',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20000331',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20000416',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_20000502',
 'LANDSAT/LE07/C02/T1_L2/LE07_016038_200

In [ ]:
#Mask clouds from images, add FLATS   
    
#     .map(maskL5sr) \
#     .map(addFLATS) \